In [2]:
import numpy as np
import json
import pandas as pd
from relation import read_json
import os
import sys
sys.path.append("..")
from src.utils.utils import sanity_check, extract_between_tags
from data.relation import read_relation_data, write_json_lists_to_file
import re

In [2]:


def write_list_to_file(filename, lst):
    with open(filename, 'w') as json_file:
        json.dump(lst, json_file)
    
def generate_final_file(json_list, filename, key):
    lst = []
    for js in json_list:
        parsed_text = extract_between_tags(js[key])
        lst.append(parsed_text)
    write_list_to_file(filename, lst)
# orgi_sample = "[CLS] Gross <e1>revenues</e1> from the <e2>selling</e2> of crude oil in 2003 reached US$11.508 billion or some 112.85 percent of the amount targeted in the 2003 state budget [SEP]"
# reph_sample = "assistant\n\nHere is a rephrased sentence with enhanced complexity while preserving the Cause-Effect relationship between the entities \"revenues\" and \"selling\":\n\n[CLS] The significant surge in <e1>revenues</e1> from crude oil in 2003, which totaled US$11.508 billion, was a direct consequence of the substantial increase in the <e2>selling</e2> of crude oil, thereby exceeding the targeted amount in the 2003 state budget by 112.85 percent [SEP]\n\nIn this rephrased sentence, I have maintained the original entities (\"revenues\" and \"selling\") and their relationship (Cause-Effect), while adding more complexity to the sentence structure and wording."

# reph_text = extract_between_tags(reph_sample)
# print(f'Rephrased: {reph_text.strip()}')
# print(orgi_sample)
# is_valid = compare_two(orgi_sample, reph_text)
# assert is_valid == True

In [49]:
rephrased_json_list = read_json('results/rephrased_relation.jsonl')
# rephrased_json_list = [js for js in rephrased_json_list if 'rephrase_response' in js]
print(len(rephrased_json_list))
error_ids = sanity_check(rephrased_json_list, key='rephrase_response')
if len(error_ids) > 0:
    print(f'problem parsing: {len(error_ids)}')
else:
    print('all good')
if len(error_ids) == 0:
    # write to file
    filename = 'results/final_rephrased_relation.json'
    key = 'rephrase_response'
    # generate_final_file(rephrased_json_list, filename, key)

7200
problem parsing: 173


In [3]:
new_text_list = read_json('results/new_generation_relation.jsonl')
new_text_list = [l for l in new_text_list if 'new_generation_response' in l]
print(len(new_text_list))

key = 'new_generation_response'
error_ids = sanity_check(new_text_list, key=key)
if len(error_ids) > 0:
    print(f'problem parsing: {len(error_ids)}')
else:
    print('all good')        
if len(error_ids) == 0:
    # write to file
    filename = 'results/final_new_sentence_relation.json'
    # generate_final_file(new_text_list, filename, key)
print(f'Errors: {len(error_ids)}')
error_ids

1064
problem parsing: 11
Errors: 11


[(52, 'parameter', 'constructor', 'constructor', 'parameter'),
 (139, 'practices', 'engineers', 'engineers', 'practices'),
 (323, '', '', '', ''),
 (359, 'club', 'homes', 'homes', 'club'),
 (496, '', '', '', ''),
 (718, 'horn', 'horn', 'valve', 'valves'),
 (794, 'outbreaks', 'outbreak', 'viruses', 'viruses'),
 (803, 'syndicate', 'syndicate', 'fraudsters', 'fraudster'),
 (863, 'band', 'band', 'outsiders', 'outsider'),
 (915, '', '', '', ''),
 (975, '', '', '', '')]

In [33]:
pseudo_label_list = read_json('results/psuedo_label_prediction_relation_fixed.jsonl')
pseudo_label_list = [l for l in pseudo_label_list if 'pseudo_label_prediction' in l]
print(len(pseudo_label_list))

accurate = 0
no_reln = 0
for js in pseudo_label_list:
    orgi_relation = js['relation_name']
    match = re.search(r'<relation>(.*?)</relation>', js['pseudo_label_prediction'])
    if match:
        if orgi_relation.strip() == match.group(1).replace(' ','').strip():
            accurate += 1
        else:
            print(f'Original: {orgi_relation};;; predicted: {match.group(1)}')
        js['predicted_relation_name'] = match.group(1)
        confidence_re= re.search(r'<confidence>(.*?)</confidence>', js['pseudo_label_prediction'])
        if confidence_re:
            js['predicted_confidence_score'] = confidence_re.group(1).strip()
        else:
            print('confidence_messedup')
    else:
        no_reln += 1
        print(f"no relation: {js['id']}::")
        pattern = r"<([A-Za-z\-]+\(((e1, e2)|(e2, e1))\))>"
        js['pseudo_label_prediction'] = re.sub(pattern, r"<relation>\1</relation>",js['pseudo_label_prediction'])

# filename = "results/psuedo_label_prediction_relation_parsed.jsonl"
# write_json_lists_to_file(filename, pseudo_label_list)
    
print(accurate/len(pseudo_label_list))
print(f'no reln: {no_reln}')

2717
Original: Message-Topic(e1,e2);;; predicted: Content-Container(e1, e2)
Original: Cause-Effect(e2,e1);;; predicted: Component-Whole(e1, e2)
Original: Product-Producer(e1,e2);;; predicted: Instrument-Agency(e1, e2)
Original: Member-Collection(e2,e1);;; predicted: Member-Collection(e1, e2)
Original: Message-Topic(e1,e2);;; predicted: Component-Whole(e2, e1)
Original: Other;;; predicted: Component-Whole(e1, e2)
Original: Entity-Destination(e1,e2);;; predicted: Content-Container(e1, e2)
Original: Product-Producer(e1,e2);;; predicted: Component-Whole(e1, e2)
Original: Entity-Origin(e1,e2);;; predicted: Content-Container(e2, e1)
Original: Other;;; predicted: Entity-Destination(e1, e2)
Original: Product-Producer(e1,e2);;; predicted: Content-Container(e1, e2)
Original: Message-Topic(e1,e2);;; predicted:  Content-Container(e1, e2) 
Original: Content-Container(e1,e2);;; predicted: Content-Container(e2, e1)
Original: Product-Producer(e1,e2);;; predicted: Component-Whole(e2, e1)
Original: Othe

In [19]:
pattern = pattern = r"<([A-Za-z\-]+\(((e1, e2)|(e2, e1))\))>"
    
    # Replace with the new format <relation>any-string(e1, e2)</relation>
result = re.sub(pattern, r"<relation>\1</relation>", "he tags e1 and e2 is:\n\n<Message-Topic(e1, e2)>\n\nThe confidence score for this relatio")
    
print(result)

he tags e1 and e2 is:

<relation>Message-Topic(e1, e2)</relation>

The confidence score for this relatio


In [35]:
match = re.search(r'\[CLS\](.*?)\[SEP\]', "assistant\n\n[CLS] A statistically significant correlation has been recently established between the heightened oviposition activity of the leaf <e1>miner</e1> and elevated concentrations of caffeine in coffee leaves, suggesting a notable association between the two variables [SEP]\n\nIn this rephrased sentence, I've maintained the same relationship between the entities <e1>egg</e1> and <e2>miner</e2>, which is Product-Producer, indicating that the leaf miner produces eggs. I've used more complex vocabulary and sentence structure to enhance the sentence's complexity while preserving the original meaning and relationship.").group(1)
match

match2 = re.search(r'\[CLS\](.*?)\[SEP\]',"[CLS] An association between increased <e1>egg</e1> laying by the leaf <e2>miner</e2> and increased caffeine levels in coffee leaves was recently recognized [SEP]",).group(1)
match, match2


(' A statistically significant correlation has been recently established between the heightened oviposition activity of the leaf <e1>miner</e1> and elevated concentrations of caffeine in coffee leaves, suggesting a notable association between the two variables ',
 ' An association between increased <e1>egg</e1> laying by the leaf <e2>miner</e2> and increased caffeine levels in coffee leaves was recently recognized ')

In [4]:
system_prompt = "You are an expert in English language task in understanding "\
    "the entitities and their relationship on the entities and their relationship."
    
system_prompt += "Please output the relationship between the two entities from the given sentence. "\
    f"The two entitites are inside <e1> and <e2> tag. Here's a sentence : \n{'question1'}\n\n"
    
system_prompt += "Please output the relationship inside a tag <relation> relation1-relation2(e1, e2) </relation>"

print(system_prompt)
            
            

You are an expert in English language task in understanding the entitities and their relationship on the entities and their relationship.Please output the relationship between the two entities from the given sentence. The two entitites are inside <e1> and <e2> tag. Here's a sentence : 
question1

Please output the relationship inside a tag <relation> relation1-relation2(e1, e2) </relation>


In [61]:


def repeated_tags_test(lst):
    no_e1 = 0
    no_e2 = 0
    for i, l in enumerate(lst):
        e1_count = l.count('<e1>')
        if e1_count == 0:
            # print(f'no e1: {i}: {l}')
            no_e1 += 1
        if e1_count > 1:
            matches = re.findall(r'<e1>(.*?)</e1>', l)
            if len(set(matches)) > 1:
                pass
                # print(f"e1 match: {i}: {matches}")
        e2_count = l.count('<e2>')
        if e2_count == 0:
            # print(f'no e2: {i}: {l}')
            no_e2 += 1
        if e2_count > 1:
            matches = re.findall(r'<e2>(.*?)</e2>', l)
            if len(set(matches)) > 1:
                pass
                # print(f'e2 match: {i}: {matches}')
    return no_e1, no_e2





def process_data(text, e1, e2):
    entities = sorted([(e1, "e1"), (e2, "e2")], key=lambda x: len(x[0]), reverse=True)
    for entity, tag in entities:
        text = re.sub(re.escape(entity), f"<{tag}>{entity}</{tag}>", text, count=1, flags=re.IGNORECASE)  # Count=1 ensures only the first occurrence is replaced
    return text

def not_e1_e2(text):
    matchese1 = re.findall(r'<e1>(.*?)</e1>', text)
    matchese2 = re.findall(r'<e2>(.*?)</e2>', text)
    counter = 0
    if not matchese1:
        return True
    return False
        

def process_no_tags(lst):
    for l in lst:
        new_gen = re.search(r'\[CLS\](.*?)\[SEP\]', l['new_generation_response']).group(1)
        # print(new_gen.group(1))
        if not_e1_e2(new_gen):
            orgi_text = l['text']
            m1 = re.findall(r'<e1>(.*?)</e1>', orgi_text)
            m2 = re.findall(r'<e2>(.*?)</e2>', orgi_text)
            e1 = m1[0].strip()
            e2 = m2[0].strip()
            l['new_generation_response'] = process_data(l['new_generation_response'], e1, e2)
    return lst

In [62]:
filename = 'results/water/water_train_new_generation_relation_finalll.jsonl'
lst = read_json(filename)
    
print(len(lst))
# noe1, noe2 = repeated_tags_test([l['new_generation_response'] for l in lst])
# noe1, noe2

# lst = process_no_tags(lst)

    
        
        
        

4176


In [63]:
c = 0
for a in lst:
    if not_e1_e2(a['new_generation_response']):
        print(a['id'])
        c += 1
c

# write_json_lists_to_file('results/water/water_train_new_generation_relation_finalll.jsonl',lst)

# print([a for a in lst if a['id']==2639])

50
85
116
176
208
294
503
505
639
830
1032
1044
1058
1078
1144
1171
1292
1412
1442
1448
1522
1557
1574
1593
1619
1695
1777
1792
1961
2018
2023
2024
2093
2101
2274
2299
2964
4047


38

In [14]:
filename = 'results/final_new_sentence_relation.json'
lst = read_json(filename)
    
print(len(lst))
repeated_tags_test(lst)

7200
e1 match: 209: ['Strawberry', 'strawberries']
e1 match: 432: ['Orange', 'oranges']
e1 match: 433: ['Vegetable', 'vegetable']
e2 match: 530: ['snipes', 'snipe']
e1 match: 677: ['Cement', 'cement']
e1 match: 1199: ['blacksmith', 'horseshoes']
e1 match: 1348: ['Discomfort', 'discomfort']
e1 match: 1478: ['depression', 'happiness']
e1 match: 2460: ['Engine', 'engine']
e1 match: 2515: ['spa', 'room']
e1 match: 2607: ['Water', 'water']
e1 match: 4262: ['Soy', 'soy']
e2 match: 4405: ['badgers', 'badger']
e1 match: 5458: ['Fog', 'rain', 'darkness', 'blowing snow']
e2 match: 5893: ['penguins', 'penguin']
e1 match: 6038: ['human body', 'central nervous system']
e1 match: 6757: ['feast', 'famine']


### ADD CLS SEP TO WATER SENTENCE

In [4]:
semeval_new_gen = read_json('results/new_generation_relation.jsonl')
semeval_new_gen_rem = read_json('results/new_generation_relation_remaining1.jsonl')



In [7]:
remaining_id_json = {js['id']: js for js in semeval_new_gen_rem}

for js in semeval_new_gen:
    if js['id'] in remaining_id_json:
        fixed = remaining_id_json[js['id']]
        js['new_generation_response'] = fixed['new_generation_response']

In [8]:
len(semeval_new_gen)

7200

In [9]:
semeval_new_gen[13]

{'id': 13,
 'text': '[CLS] This <e1>dishwasher</e1> has the tallest <e2>tub</e2> available* and allows you to wash more than ever before [SEP]',
 'label': 1,
 'relation_name': 'Component-Whole(e2,e1)',
 'new_generation_response': 'assistant\n\nHere is a new sentence with the same entities and relationship:\n\n[CLS] The <e2>tub</e2> of this <e1>dishwasher</e1> is designed to maximize water capacity [SEP]\n\nIn this sentence, the relationship between the entities is still Component-Whole(e2,e1), where the tub is a component of the dishwasher.'}

In [10]:
write_json_lists_to_file('results/new_generation_relation.jsonl',semeval_new_gen)

### merge water new